In [1]:
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras import optimizers
from keras import applications
from keras.models import Model

Using TensorFlow backend.


In [4]:
import os
import math
from keras.utils import np_utils

In [9]:
print(os.chdir("/home/ubuntu/modeldata/"))

None


In [11]:
# dimensions of our images.
img_width, img_height = 150, 150
os.chdir("/home/ubuntu/modeldata")
train_data_dir = 'data/train'
validation_data_dir = 'data/validation'

In [12]:
def fcount(path):
    count1 = 0
    for root, dirs, files in os.walk(path):
            count1 += len(dirs)
    return count1

In [13]:
train_labels = fcount("/home/ubuntu/modeldata/data/train/")
validation_labels= fcount("/home/ubuntu/modeldata/data/validation/")

In [18]:
##preprocessing
# used to rescale the pixel values from [0, 255] to [0, 1] interval
datagen = ImageDataGenerator(rescale=1./255)
batch_size = 30

In [19]:
train_generator_bottleneck = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)

validation_generator_bottleneck = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)

Found 210 images belonging to 3 classes.
Found 90 images belonging to 3 classes.


In [20]:
epochs = 20
train_samples = train_labels*70
validation_samples = validation_labels*30

In [21]:
model_vgg = applications.VGG16(include_top=False, weights='imagenet')

In [22]:
bottleneck_features_train = model_vgg.predict_generator(train_generator_bottleneck, train_samples // batch_size)
np.save(open('models/bottleneck_features_train.npy', 'wb'), bottleneck_features_train)

In [23]:
bottleneck_features_validation = model_vgg.predict_generator(validation_generator_bottleneck, validation_samples // batch_size)
np.save(open('models/bottleneck_features_validation.npy', 'wb'), bottleneck_features_validation)

In [24]:
train_data = np.load(open('models/bottleneck_features_train.npy', 'rb'))
train_labels = np.array([0] * (train_samples // 3) + [1] * (train_samples // 3) + [2] * (train_samples // 3))

validation_data = np.load(open('models/bottleneck_features_validation.npy', 'rb'))
validation_labels = np.array([0] * (validation_samples // 3) + [1] * (validation_samples // 3) + [2] * (validation_samples // 3))

In [25]:
# Convert 1-dimensional class arrays to 3-dimensional class matrices (one hot vector encoding)
train_labels = np_utils.to_categorical(train_labels, 3)
validation_labels = np_utils.to_categorical(validation_labels, 3)

In [26]:
model_top = Sequential()
model_top.add(Flatten(input_shape=train_data.shape[1:]))
model_top.add(Dense(256, activation='relu'))
model_top.add(Dropout(0.5))
model_top.add(Dense(3, activation='softmax'))

model_top.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [27]:
model_top.fit(train_data, train_labels,
            epochs=epochs, 
            batch_size=batch_size,
            validation_data=(validation_data, validation_labels))

Train on 210 samples, validate on 90 samples
Epoch 1/20
210/210 [==============================] - 8s - loss: 1.5975 - acc: 0.5095 - val_loss: 0.6554 - val_acc: 0.7333
Epoch 2/20
210/210 [==============================] - 0s - loss: 0.6528 - acc: 0.6667 - val_loss: 0.5801 - val_acc: 0.6889
Epoch 3/20
210/210 [==============================] - 0s - loss: 0.5263 - acc: 0.7667 - val_loss: 0.4837 - val_acc: 0.8222
Epoch 4/20
210/210 [==============================] - 0s - loss: 0.5831 - acc: 0.6952 - val_loss: 0.4866 - val_acc: 0.8556
Epoch 5/20
210/210 [==============================] - 0s - loss: 0.4373 - acc: 0.8286 - val_loss: 0.4314 - val_acc: 0.7778
Epoch 6/20
210/210 [==============================] - 0s - loss: 0.4556 - acc: 0.7905 - val_loss: 0.395

In [28]:
model_top.save_weights('models/Test_bottleneck_30_epochs.h5')

In [29]:
model_2 = applications.VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
#model_vgg = applications.VGG16(weights='imagenet', include_top=False)

In [30]:
model_2.output_shape[1:]

(4, 4, 512)

In [31]:
top_model = Sequential()
top_model.add(Flatten(input_shape=model_2.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(3, activation='softmax'))
top_model.load_weights('models/Test_bottleneck_30_epochs.h5')

In [32]:
#model_vgg.add(top_model)
model = Model(inputs = model_2.input, outputs = top_model(model_2.output))

In [33]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

In [34]:
for layer in model.layers[:15]:
    layer.trainable = False

In [35]:
# compile the model with a SGD/momentum optimizer and a very slow learning rate.
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [36]:
# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size
        )

validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size
        )

Found 210 images belonging to 3 classes.
Found 90 images belonging to 3 classes.


In [37]:
batch_size

30

In [38]:
# fine-tune the model
model.fit_generator(
    train_generator,
    steps_per_epoch=train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_samples // batch_size)

Epoch 1/20
7/7 [==============================] - 7s - loss: 0.2716 - acc: 0.9143 - val_loss: 0.2332 - val_acc: 0.9111
Epoch 2/20
7/7 [==============================] - 6s - loss: 0.2353 - acc: 0.9095 - val_loss: 0.1822 - val_acc: 0.9778

In [39]:
model.save_weights('models/Test_finetuning_30epochs_vgg.h5')

In [41]:
import glob
import os
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
cwd = os.getcwd()
fname = glob.glob("/home/ubuntu/modeldata/data/train/cats/*.jpg")
for i in range(len(fname)):    
    img = load_img(fname[i],False, (img_width, img_height))
    x = img_to_array(img)
    prediction = model.predict(x.reshape((1,img_width, img_height,3)),batch_size=32, verbose=0)
    print(np.round(prediction))

[[ 0.  1.  0.]]
[[ 0.  1.  0.]]
[[ 0.  1.  0.]]
[[ 1.  0.  0.]]
[[ 1.  0.  0.]]
[[ 0.  1.  0.]]
[[ 1.  0.  0.]]
[[ 0.  1.  0.]]
[[ 1.  0.  0.]]
[[ 0.  1.  0.]]
[[ 1.  0.  0.]]
[[ 1.  0.  0.]]
[[ 1.  0.  0.]]
[[ 1.  0.  0.]]
[[ 0.  1.  0.]]
[[ 1.  0.  0.]]
[[ 1.  0.  0.]]
[[ 0.  1.  0.]]
[[ 1.  0.  0.]]
[[ 0.  1.  0.]]
[[ 1.  0.  0.]]
[[ 0.  1.  0.]]
[[ 1.  0.  0.]]
[[ 1.  0.  0.]]
[[ 0.  1.  0.]]
[[ 1.  0.  0.]]
[[ 0.  1.  0.]]
[[ 1.  0.  0.]]
[[ 1.  0.  0.]]
[[ 1.  0.  0.]]
[[ 0.  1.  0.]]
[[ 1.  0.  0.]]
[[ 1.  0.  0.]]
[[ 1.  0.  0.]]
[[ 1.  0.  0.]]
[[ 1.  0.  0.]]
[[ 0.  1.  0.]]
[[ 0.  1.  0.]]
[[ 0.  1.  0.]]
[[ 0.  1.  0.]]
[[ 0.  1.  0.]]
[[ 1.  0.  0.]]
[[ 1.  0.  0.]]
[[ 1.  0.  0.]]
[[ 1.  0.  0.]]
[[ 1.  0.  0.]]
[[ 0.  1.  0.]]
[[ 0.  1.  0.]]
[[ 1.  0.  0.]]
[[ 1.  0.  0.]]
[[ 1.  0.  0.]]
[[ 1.  0.  0.]]
[[ 1.  0.  0.]]
[[ 1.  0.  0.]]
[[ 1.  0.  0.]]
[[ 0.  1.  0.]]
[[ 0.  1.  0.]]
[[ 0.  1.  0.]]
[[ 1.  0.  0.]]
[[ 1.  0.  0.]]
[[ 1.  0.  0.]]
[[ 1.  0.  0.]]
[[ 0.  1